# Problem Statement

Let $(a_n)_{n\ge0}$ be a sequence defined by 

$$a_0 = 3, \qquad
    a_{n+1} =
    \begin{cases}
        a_n + 1,            & \text{if } a_n \text{ is a triangular number}, \\[4pt]
        2a_n - a_{n-1} + 1, & \text{otherwise.}
    \end{cases}$$

Define the triangular numbers

$T_j = \frac{j(j+1)}{2}, j\in\mathbb{N}$. Let
$(a_{n_k})_{k\ge1}$ denote the subsequence of $(a_n)$ consisting of
those terms that are triangular. The indices $(n_k)$ satisfy
$a_{n_k} = T_{j_k}$ for some $(j_k) \in \mathbb{N}$, with $n_1=0$ and
$j_1=2$.

Given $n_{10}=2964$, find $n_{70}$, the index of the $70$-th
triangular number appearing in the sequence.

In [2]:
import math
import sympy as sp


def T(n: int) -> int:
    return n * (n + 1) // 2


def is_tri(x: int) -> bool:
    d = 1 + 8 * x
    s = int(math.isqrt(d))
    return s * s == d


def minimal_m_for_j(j: int):
    """
    Let be a minimum m>0: T_j + T_m triangular,
    Factor approach: (w-v)(w+v) = 4 j (j+1),
    x=(w-v)/2, y=(w+v)/2, xy = j(j+1), v = y-x = 2m+1 odd.
    """
    N = j * (j + 1)

    # pgcd(j, j+1)=1 in this way, divisors of N come from the pairs d|j and e|(j+1): y = d*e, x = N // y.
    dj = sp.divisors(j)
    dp = sp.divisors(j + 1)

    best_v = None

    for d in dj:
        for e in dp:
            y = d * e
            x = N // y
            v = y - x
            # Should be v > 1 and odd: plus, x,y must be of opposite parity (that is, v odd is enough).
            if v > 1 and (v % 2 == 1):
                if best_v is None or v < best_v:
                    best_v = v

    if best_v is None:
        raise RuntimeError("(x,y) not found for j={}".format(j))

    m = (best_v - 1) // 2

    # If we want to return also j' (the index of the next triangular number seen):
    Tr = T(j) + T(m)
    r = (int(math.isqrt(1 + 8 * Tr)) - 1) // 2
    assert is_tri(Tr) and T(r) == Tr
    return m, r


def index_of_kth_triangular(k: int) -> int:
    """
    n_k is the index such that a_{k} = T_{n_k} is the k-th triangular number
    which can be written as a sum of two triangular numbers.
    Initial condition: a_0 = T_2, cad k=1 -> n_1 = 0.
    """
    if k < 1:
        raise ValueError("It should be k >= 1")
    n = 0  # n_1 = 0
    j = 2  # T_2 = 3
    for _ in range(1, k):  # It not enter to the loop if k=1.
        m, j_next = minimal_m_for_j(j)
        n += m
        j = j_next
    return n

assert index_of_kth_triangular(10) == 2964
print(index_of_kth_triangular(70))

6795261671274


# Main Theorem

$$\boxed{
        \begin{aligned}
            n_1         & = 0, \quad j_1 = 2,  \\[4pt]
            m_k         & = m(j_k)
            = \min_{\substack{xy=j_k(j_k+1)    \\ y-x\ \text{odd},\ y-x>1}}
            \frac{y-x-1}{2},                   \\[6pt]
            n_{k+1}     & = n_k + m_k,         \\[4pt]
            T_{j_{k+1}} & = T_{j_k} + T_{m_k}.
        \end{aligned}
    }$$

*Proof.* By Lemma 2.3, the total increase between triangular terms is
$T_{m_k}$. By Lemma 3.1, $T_{j_k} + T_{m_k}$ is triangular iff
$xy = j_k(j_k+1)$ with $y - x$ odd. Choosing minimal $y-x$ gives the
earliest next triangular term. 

Please see also triangular.tex for lemmas and demonstrations.